In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? Зачем он нужен. Используя этот товар мы смещяем качество рекомендаций. В какую сторону?  Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре.

In [27]:
#999999 - это индекс всех товаров не вошедшив в топ 5000. если размер товаров не ограничивать то размер
# данных такой (2499, 86865)

In [42]:
#'Спсиок рекомендаций, если не ограничиваться топ 5000:

def str_to_list(data):    
    res = []
    for i in data.strip('][').split():
        res.append(int(i.rstrip(',')))
    return res

result_not5000 = pd.read_csv('not5000_predictions_basic.csv') # закгрузка predict с семианара

for col in result_not5000.iloc[:,1:]:
    result_not5000[col] = result_not5000.apply(lambda row: str_to_list(row[col]), axis=1)
result_not5000.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12610955, 9803248, 9798469, 1946014, 7443299]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 13671834, 12487387]","[1082185, 13671834, 12487387, 12171883, 8091337]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1373403, 9880531, 94548, 13769223, 15830550]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]"


In [43]:
for col in result_not5000.iloc[:,2:]:
    print(col, end=': ')
    print(result_not5000.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean())

random_recommendation: 0.0007835455435847209
popular_recommendation: 0.15523996082272082
itemitem: 0.15406464250734386
cosine: 0.1295788442703217
tfidf: 0.12291870714985192
own_purchases: 0.21909892262487482


In [ ]:
#Результат второго задания:

# random_recommendation: 0.0005876591576885406
# popular_recommendation: 0.15523996082272082
# itemitem: 0.13692458374142857
# cosine: 0.13290891283055686
# tfidf: 0.1389813907933383
# own_purchases: 0.17969311132876015

Точность упала, но не на всех способах рекомендации. Во первых данных значительно больше (2499, 86865) против (2499, 5001), а вить мы это для экономии памяти делали. Товар 999999 символизирует все 86865 - 5000 = 81865 товаров не в топ 5000, я конечно не спец, но мне кажется представив 81865 как один мы проиграли в рекомендациях.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [61]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    item_id = np.array(items_weights['item_id'])
    recs = np.random.choice(item_id, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

Сделайте предсказания

In [71]:
%%time

# your_code
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.loc[items_weights['sales_value'] < 1] = 1
items_weights['weight'] = items_weights['sales_value'].apply(np.log)
items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()
items_weights.head()

Wall time: 160 ms


,item_id,sales_value,weight
0,25671,20.94,0.000013
1,1,1.00,0.000000
2,26093,1.59,0.000002
3,26190,1.54,0.000002
4,26355,1.98,0.000003


In [72]:
weighted_random_recommendation(items_weights)

[7441727, 10455984, 960725, 9837585, 876806]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [25]:
result = pd.read_csv('predictions_basic.csv') # закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1077429, 13008386, 9837278, 979239, 990483]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[956867, 9878738, 1171012, 976394, 5129575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [35]:
for col in result.iloc[:,1:]:
    result[col] = result.apply(lambda row: str_to_list(row[col]), axis=1)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1077429, 13008386, 9837278, 979239, 990483]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[956867, 9878738, 1171012, 976394, 5129575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [79]:
result['random_waihgt'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

In [80]:
# your_code

for col in result.iloc[:,2:]:
    print(col, end=': ')
    print(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean())

random_recommendation: 0.0007835455435847209
popular_recommendation: 0.15523996082272082
itemitem: 0.15406464250734386
cosine: 0.1295788442703217
tfidf: 0.12291870714985192
own_purchases: 0.21909892262487482
random_waihgt: 0.001958863858961803


In [ ]:
# Луший ItemItemRecommender(K=1, num_threads=4) трюк


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [83]:
# your_code

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
data_test.loc[~data_test['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix_test[user_item_matrix_test > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix_test = user_item_matrix_test.astype(float) # необходимый тип матрицы для implicit

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

userids_test = user_item_matrix_test.index.values
itemids_test = user_item_matrix_test.columns.values

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [84]:
user_item_matrix.shape

(2499, 5000)

In [124]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 999999, 91419..."
1,3,"[999999, 851057, 872021, 878302, 879948, 90963..."


In [127]:
%%time
for K in range(1, 15): 
    model = ItemItemRecommender(K=K, num_threads=4) # K - кол-во ближайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

    result[f'K-{K}'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])


Wall time: 12.9 s


In [128]:
result.head()

,user_id,actual,K-1,K-2,K-3,K-4,K-5,K-6,K-7,K-8,K-9,K-10,K-11,K-12,K-13,K-14
0,1,"[821867, 834484, 856942, 865456, 999999, 91419...","[10455984, 12172240, 12810393, 15926844, 12324...","[10455984, 12172240, 12810393, 15926844, 12324...","[981760, 6534178, 12324948, 15926844, 12810393]","[981760, 1127831, 1098066, 854852, 866211]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]","[981760, 1127831, 1098066, 826249, 6534178]"
1,3,"[999999, 851057, 872021, 878302, 879948, 90963...","[12810393, 13416351, 13945244, 15596279, 13842...","[12810393, 13416351, 13945244, 15596279, 13842...","[981760, 13416351, 13842214, 13945244, 15596279]","[981760, 995242, 908531, 840361, 961554]","[981760, 995242, 1029743, 840361, 961554]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 961554]","[981760, 995242, 840361, 1029743, 961554]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 1004906]","[981760, 995242, 840361, 1029743, 1004906]"
2,6,"[920308, 926804, 999999, 1017061, 1078346, 112...","[12731436, 12731544, 13511722, 15926712, 13003...","[12731436, 12731544, 13511722, 15926712, 13003...","[12731436, 12731544, 13511722, 15926712, 13003...","[1058997, 1098066, 916122, 860776, 1005186]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 854852, 1106523, 923746]","[1098066, 826249, 854852, 860776, 923746]","[1098066, 826249, 854852, 860776, 923746]","[1098066, 826249, 854852, 860776, 923746]","[1098066, 826249, 854852, 860776, 923746]","[1098066, 826249, 854852, 860776, 923746]","[1098066, 826249, 860776, 854852, 916122]","[1098066, 826249, 860776, 854852, 916122]","[1098066, 826249, 860776, 854852, 923746]"
3,7,"[840386, 889774, 898068, 909714, 999999, 95347...","[12384775, 12731436, 13511722, 15511891, 12810...","[12384775, 12731436, 13511722, 15511891, 12810...","[981760, 12731436, 12810393, 13511722, 15511891]","[981760, 1098066, 883404, 916122, 840361]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 840361, 883404, 6534178]","[981760, 1098066, 840361, 883404, 6534178]","[981760, 1098066, 840361, 883404, 6534178]","[981760, 1098066, 840361, 883404, 6534178]","[981760, 1098066, 840361, 883404, 6534178]","[981760, 840361, 1098066, 883404, 6534178]","[981760, 840361, 1098066, 883404, 6534178]","[981760, 840361, 1098066, 6534178, 883404]","[981760, 840361, 1098066, 6534178, 904360]"
4,8,"[835098, 872137, 910439, 924610, 999999, 10412...","[12810393, 13115903, 13190294, 15596515, 13189...","[12810393, 13115903, 13190294, 15596515, 13189...","[12810393, 13115903, 13190294, 15596515, 13189...","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 866211, 979707, 13190294, 15596515]","[904360, 866211, 979707, 13190294, 15596515]","[904360, 866211, 1096036, 979707, 908531]","[904360, 866211, 1096036, 979707, 908531]","[904360, 866211, 1096036, 979707, 908531]","[904360, 866211, 1096036, 979707, 908531]","[904360, 866211, 1096036, 979707, 908531]","[904360, 866211, 1096036, 908531, 979707]","[904360, 866211, 1096036, 979707, 908531]"


In [129]:
for col in result.iloc[:,2:]:
    print(col, end=': ')
    print(result.apply(lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean())

K-1: 0.06358145608880222
K-2: 0.0620633365981069
K-3: 0.05990042442050322
K-4: 0.04204211557296795
K-5: 0.03408423114593552
K-6: 0.031929480901077514
K-7: 0.03143976493633706
K-8: 0.03173359451518134
K-9: 0.030656219392752336
K-10: 0.030460333006856143
K-11: 0.030362389813908056
K-12: 0.030068560235063782
K-13: 0.029872673849167603
K-14: 0.02967678746327142


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.